In [12]:
# quote URL param
from urllib.parse import quote

In [27]:
x_range=[-1, 7]
y_range=[-1, 17]
plot_height=700
plot_width=900

In [36]:
import plotly.graph_objs as go

f = go.FigureWidget(data=[{'x': x_range,
                           'y': y_range,
                           'mode': 'markers',
                           'marker': {'opacity': 0}}], # invisible trace to init axes and to support autoresize
                    layout={'width': plot_width, 'height': plot_height})
f.update_xaxes(gridcolor='rgba(220, 220, 220, 0.5)')
f.update_yaxes(autorange="reversed", gridcolor='rgba(220, 220, 220, 0.5)')

import random
def get_url(x_range, y_range, width, height):
    x_min = x_range[0]
    x_max = x_range[1]
    
    y_min = min(y_range)
    y_max = max(y_range)

    stilts_url  = 'http://andromeda.star.bristol.ac.uk:8081/plotserv/plot/img/plot2plane'
    stilts_url += '&xpix={}&ypix={}&in=hrd-100pc.fits&insets=0,0,0,0'.format(width, height)
    stilts_url += '&xmin={}&xmax={}&ymin={}&ymax={}'.format(x_min, x_max, y_min, y_max)
    stilts_url += '&yflip=true&icmd=select+astrometric_excess_noise%3C1&icmd=select+phot_bp_rp_excess_factor%3CpolyLine(bp_rp%2C-0.56%2C1.307%2C0.03%2C1.192%2C1.51%2C1.295%2C4.31%2C1.808)&layer1=mark&x1=bp_rp&y1=phot_g_mean_mag%2B5*log10(parallax/100)&shading1=density&densemap1=plasma'
 
    stilts_url += '?sessionId={}'.format(random.randint(0, 1e15))
    return 'http://alasky.u-strasbg.fr/cgi/JSONProxy?url={}'.format(quote(stilts_url))
    
f.layout.images = [go.layout.Image(
    source = get_url(x_range, y_range, plot_width, plot_height),
    xref = "x",
    yref = "y",
    x = x_range[0],
    y = y_range[1],
    sizex = x_range[1] - x_range[0],
    sizey = y_range[1] - y_range[0],
    sizing = "stretch",
    layer = "below")]  

from PIL import Image
import requests
from io import BytesIO



def update_ds_image(layout, x_range, y_range, plot_width, plot_height):
    img = f.layout.images[0]

    # Update with batch_update so all updates happen simultaneously
    with f.batch_update():
        response = requests.get(get_url(x_range, y_range, plot_width, plot_height))
        img.source = Image.open(BytesIO(response.content))
#        img.source = get_url(x_range, y_range, plot_width, plot_height) # ==> this does not work properly and makes the plot flicker

        img.sizex = x_range[1] - x_range[0]
        img.sizey = y_range[1] - y_range[0]
        img.x = x_range[0]
        img.y = y_range[1]

# Install callback to run exactly once if one or more of the following properties changes
#  - xaxis range
#  - yaxis range
#  - figure width
#  - figure height
f.layout.on_change(update_ds_image, 'xaxis.range', 'yaxis.range', 'width', 'height')

In [37]:
f

FigureWidget({
    'data': [{'marker': {'opacity': 0},
              'mode': 'markers',
              'type': …